<a href="https://colab.research.google.com/github/mzk8888/AI/blob/main/qa4u3_day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

それでは量子アニーリングを利用したアプリ開発、サービス開発の実践に向けて初めていきましょう。
量子アニーリングを用いた、それこそ実機を用いた解析には必須のライブラリのインストールです。pythonによるプログラミング開発ではpip installで必要なライブラリを準備してから始めます。
 （こうした呪文であらかじめ頭のいい人がプログラムしたものを呼び出すことができます） 今回の場合ですと、dwave-ocean-sdkをインストールすることになります。

In [ ]:
pip install dwave-ocean-sdk

ご自身のローカル環境へインストールする際も同様の手順で差し支えありません。 （ただし環境によってうまくインストールできるかどうかはわかりませんので、適宜ご対応いただければと思います） これでD-Wave Systemsの量子アニーリングマシンを利用することができる準備が整いました。 たったこれだけの準備だてでOKです。

## 量子アニーリングマシンの目的

量子アニーリングマシンは、「組合せ最適化問題」を解くために生まれた、まさに未来を感じさせるマシンです！  
最適化問題と聞くと「うわ、数学だ…」と身構えるかもしれません。でも実は、これって中学・高校でやった「最大値・最小値を求めなさい」といった問題の発展版なんです。  

たとえば、将来の収入を予測する関数があれば、それを最大化したいですよね。逆に、将来の苦労を予測する関数なら、それを最小化して楽な人生を送りたい！  
こういった考え方を量子アニーリングマシンは、自然の力を利用して実現してくれるんです。

## 量子アニーリングマシンが挑む「関数の最小化」

量子アニーリングでは、次のような関数の最小値と、そのときのベストな「x」を探してくれます。

\[
E({\bf x}) \sum_{i=1}^N \sum_{j=1}^N Q_{ij}x_ix_j
\]

「シグマ記号」と聞いて「なんかカッコイイ！でも難しそう…」と思った方、大丈夫です！要するに「全部足し算してるだけ」です。添え字とか記号は「選択肢がいくつかあるよ」ということを端的に表しているだけなんですよ。  

こういった関数を最小化する問題を、「制約なし2値の2次計画問題」と呼びます。英語だと **QUBO** (Quadratic Unconstrained Binary Optimization) といいます。  
この最小化の対象となる関数は「コスト関数」と呼ばれ、量子アニーリングマシンにとっては解決すべきお題のようなものなんです。

## 量子アニーリングマシンに命令を送る方法

量子アニーリングマシンに解かせたい問題を指示するには、「QUBO行列」というものを作って送ります。この行列は、問題の要素をギュッと詰め込んだ指令書のようなもの。量子アニーリングマシンは、この指令書を受け取って計算を始めます。  

ちなみに、このQUBO行列を作るときに使える便利ツールが **Python** のライブラリ「numpy」です。これを使えば、行列もサクッと準備できます！

## D-Wave Leapに登録してみよう！

量子アニーリングマシンを実際に使うには、まずアカウント登録が必要です！  
以下のリンクにアクセスして、メールアドレスや所属などを入力してアカウントを作りましょう。  

[D-Wave Leap](https://cloud.dwavesys.com/leap/)  
https://cloud.dwavesys.com/leap/

登録が済んだら、左下に表示されている **API Token** をコピーしておきましょう。このトークンが、量子アニーリングマシンとやり取りをするためのカギになります。  

すでにD-Wave Leapの試用期間が終わってしまった方も大丈夫！**QA4U3** に参加登録すれば、新しい無料マシンタイムが手に入るので安心してくださいね。

## 簡単な問題を解かせてみよう！

アカウント登録が完了したら、いよいよ量子アニーリングマシンに問題を投げかけてみましょう！  
問題を送るときは、先ほど紹介した **QUBO行列** を作って、それをカナダにある量子アニーリングマシンに送信するだけです。  

Python の **numpy** を使えば、行列作成はとっても簡単。まずはそうしたライブラリを利用するための準備です。importしましょう！


In [ ]:
import numpy as np

次に簡単な例として、10×10行列によるQUBOを考えてみます。 np.random.randn()は、平均0、分散1のガウス分布に従う乱数を生成するという関数です。
それをN**2=100個作ったのち、reshape(N,N)として、10×10の行列の形にします。

In [ ]:
#変数の数を10個に指定します
N = 10
#QUBO行列を作ります
QUBO = np.random.randn(N**2).reshape(N,N)
#細かいことですが、対称行列にしましょう。
QUBO = (QUBO + QUBO.T)/2

ここでQUBO+QUBO.Tというところで、.Tというコマンドで転置をとっています。 行列の形を対称な形にするためです（細かいことですがたまに忘れます） どんな値になっているのか調べたい人はprint等を利用して調べてみると良いでしょう。

さてこうした下準備が終わりましたら、量子アニーリングマシンに投入してみましょう。 これだけで準備完了です。 まずは先ほど取得したAPI tokenを準備します。

In [ ]:
token = 'XX'  # 個人のAPI tokenを使用

続けて量子アニーリングマシンを呼び出す関数をいくつか用意します

In [ ]:
from dwave.system import DWaveSampler, EmbeddingComposite

**from dwave.system**とあるのはOcean SDKのdwave.systemの中からということです。

**import DWaveSampler**でDWaveSamplerという関数を呼び出しており、 これは量子アニーリングを行うQPUチップを利用する関数です。

**import EmbeddingComposite**でEmbeddingCompositeという関数を呼び出しています。 これは用意したQUBO行列をQPUチップに埋め込むための関数です。

In [ ]:
dw_sampler = DWaveSampler(solver='Advantage_system4.1', token=token)

ここでsolver=というオプションで、どの量子アニーリングマシンを使うのかを指定することができます。 最新式の量子アニーリングマシンはD-Wave Advantageであり、それを利用する場合には solver = 'Advantage_system4.1'としましょう。 前の形式の量子アニーリングマシンであるD-Wave 2000Qを利用する場合には solver = 'DW_2000Q_6'としましょう。 さらに次世代の量子アニーリングマシン用のプロトタイプのチップも利用できます。 その場合には、 solver = 'Advantage2_prototype2.6' とします。

次におまじない的ですが、QUBO行列をQPUに埋め込むための作業を行うために EmbeddingCompositeという機能を付与します。

In [ ]:
sampler = EmbeddingComposite(dw_sampler)

これでみなさんのQUBO行列をなんでも埋め込めるようになりました。 回路の限界はありますが、できるかぎりQUBO行列をQPUチップに載せてくれます。 それでは早速量子アニーリングマシンに問題を投入しましょう。 以下のコマンドでOKです。その際にnum_readsで何回量子アニーリングを実行するのかを指定しましょう。

In [ ]:
sampleset = sampler.sample_qubo(QUBO, num_reads=10)

このコマンドを実行すると、カナダに問題が送信されます。順次彼方では量子アニーリングマシンが他のジョブとともに次々に組合せ最適化問題を解き、その結果を記録しています。 私たちはその結果を取得する必要があります。 そのために行うのが、samplesetの実行です。 例えばどんな結果が出てきたのかをみたいときにはsampleset.recordなどを打つと良いでしょう。

In [ ]:
print(sampleset.record)

いくつか数値が並んでいますが、最初にある0,1のリスト（たくさんの数字の要素が並んでカッコでまとめられているもの）が結果です。
次にある数値がエネルギー、これはコスト関数の値が実際どのような値になるのかがわかると言うものです。次に出てくる整数値は何回その結果が出てきたと言うものです。最後にある数値はチェーンブレイクしている割合です。詳しくはのちに回しますが、要するに答えの信頼が崩れている度合いです。

## モードを切り替える

実行する目的によってはanswer_modeを変えると良いでしょう。今の場合、同じ結果が何回出たのか頻度がわかりやすい形です。num_reads回実行したら、その順番でどんな結果が出たのかが見たいときもあります。その場合にはanswer_mode="raw"とします。

In [ ]:
sampleset = sampler.sample_qubo(QUBO, num_reads=100, answer_mode = "raw")

これで結果を出力すると同じ答えが出ていたとしても、まとめずに１つの結果として出力されます。


In [ ]:
print(sampleset.record)

このanswer_modeですと、コスト関数がどのような値を取っていたのか、その頻度をプロットしたヒストグラムを描くのに便利です。エネルギーだけのリスト（numpy.array）を取得するためにはsampleset.data_vectors["energy"] を利用します。

In [ ]:
import matplotlib.pyplot as plt
ene = sampleset.data_vectors["energy"]
plt.hist(ene, bins=20)
plt.show()

それではこれを残しておいて、別のQPUでも試してみて、性能を比較してみましょう。 このノートブックではAdvantage6.1とAdvantage2_prototype1.1を比較してみます。

In [ ]:
dw_sampler2 = DWaveSampler(solver='Advantage2_prototype2.6', token=token)
sampler2 = EmbeddingComposite(dw_sampler2)
sampleset2 = sampler2.sample_qubo(QUBO, num_reads=100, answer_mode = "raw")

D-Waveマシンを使う関数群については一気に利用してみました。 それぞれ変数名等を2と付してあります。

In [ ]:
ene2 = sampleset2.data_vectors["energy"]
plt.hist(ene2, bins=20)
plt.show()

おそらく結果が異なる傾向にあると思います。 せっかくですので重ねて比較してみましょう。 range=(min,max)で範囲を揃えて描くと綺麗に仕上がります。

In [ ]:
plt.hist(ene, bins=20, alpha=0.5,range=(-9.5,-7.5))
plt.hist(ene2, bins=20, alpha=0.5,range=(-9.5,-7.5))
plt.show()

これを見ますと（青が先に用意したもの、オレンジが後に用意したものです） Advantage2が優勢で、より低いエネルギーを持つ答えを得ることに成功しているのがわかります。

## わかりやすい例題で量子アニーリング！

それではQUBOを今回はいじりまして、少し面白い例題に取り組みたいと思います。
去年ノーベル物理学賞を受賞したホップフィールドで有名なホップフィールドネットワークです。
ホップフィールドネットワークは画像に代表される0と1のパターン画像を想起する機能を持ちます。
まずもとになる画像データを呼び出しましょう。
それにはscikit-learnと呼ばれる機械学習ライブラリ群を利用します。

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits

ここでfrom sklearn.datasetからいくつかある公開データセットのうち、import load_digitsを利用して手書き文字のデータセットを読み込みます。

In [ ]:
digit=load_digits()

digitという変数に手描き文字のデータが格納されます。

In [ ]:
K = 5
ind = np.random.choice(len(digit.data), K, replace=False)

K=2と指定して2個のデータを読み込むことにします。あとで変えられるようにKにデータの個数を入れて指定できるようにしています。
random.choiceで、全てのデータのうちK個だけ選び出します。何番目のデータを使うのかという数字を返すのがこの関数です。
（replace=Falseで重複なしの数字が出てきます。同じ数字が出ない）

In [ ]:
data = digit.data[ind]

その数字のリスト（np.array形式）を入れるとその数で指定されたデータだけ抜き出します。
dataを確認してみましょう。
その際にイジング模型に即した-1と1ないしは0と1の2値画像に直します。

In [ ]:
binary_data = np.where(data > np.mean(data), 1, 0)
Ising_data = np.where(data > np.mean(data), 1, -1)

これを画像として眺めてみましょう。
load_digitsで読み込まれるのは8×8のサイズの画像ですので合計64個の数字を持ったベクトルの形式で保存されています。
こちらを縦横8のサイズにreshapeした上で、画像を表示してみましょう。
plt.imshow()を利用します。

In [ ]:
N = 8
for k in range(K):
  plt.imshow(binary_data[k,:].reshape(N,N))
  plt.show()

何やら手書き文字として認識できる数字が見られるかと思います。
その数字をイジング模型に埋め込み、イジング模型に合わせて最適化したものから、画像が再現されることを見てみましょう。

それではホップフィールド模型の相互作用行列$J_{ij}$と磁場ベクトル$h_i$を組み立てていきます。
まずホップフィールド模型は磁場は必要ありません。
どちらかというと相互作用行列で画像などのパターンを記憶します。
とりわけ他の画素値（スピン）とどのような関係にあったのか、隣同士のスピンとの相互作用によって決まるというわけです。
そこで

$J_{ij} = -\frac{1}{K} \sum_{p=1}^K S^p_i S^p_j$

と相互作用行列$J_{ij}$に$S_i$画素値の積を入れていきます。
異なるパターンは上付き添え字で区別することにします。


In [ ]:
import dimod

前回同様にイジング模型をQUBO形式に変形するdimodの機能を利用する準備をして、
上記の数式の通りにJijにパターンを書き込みます。

In [ ]:
Jmat = np.zeros([N*N,N*N])
for k in range(K):
  for i1 in range(N):
    for j1 in range(N):
      for i2 in range(N):
        for j2 in range(N):
          Jmat[i1+N*j1,i2+N*j2] -= Ising_data[k,i1+N*j1]*Ising_data[k,i2+N*j2]/K

hvec = np.zeros(N*N)

上記と同じことをやるプログラムは以下のように簡潔に書くことができます。

In [ ]:
Jmat = np.zeros([N*N,N*N])
for k in range(K):
  Jmat -= np.outer(Ising_data[k,:],Ising_data[k,:])/K

hvec = np.zeros(N*N)

ここで大事なことは複数のパターンを$J_{ij}$に書き込んでいることです。
また-1と+1の表示の方を利用しています。これは同じ画素が並んでいるかどうかを調べるのにイジング形式の方が向いているからです。
ここでできた相互作用行列と磁場ベクトルをQUBOに直してもらいましょう。

In [ ]:
model = dimod.BinaryQuadraticModel(hvec, Jmat, 0.0, vartype='SPIN')
qubo, offset = model.to_qubo()

そうしてできたQUBOを量子アニーリングマシンに投げます。

In [ ]:
sampleset = sampler.sample_qubo(qubo, num_reads=10)

num_reads=10などと指定して、いくつかの結果をサンプリング（実際に試しに出力する）してみましょう。
その結果を同じように画像として出力して出来栄えを見てみることにします。

In [ ]:
for k in range(len(sampleset.record)):
  res_image = sampleset.record[k][0]
  plt.imshow(res_image.reshape(N,N))
  plt.show()

うまく出力できていることが見えるかと思います。
複数の記憶を$J_{ij}$は記憶することができる。
その記憶にもどついて想起することができる。
現代風に言えば生成ができるということになります。
幾つのパターンまで覚えられるのか、ソルバーごとに違いはないのか、など色々と気になるかと思います。

## ボルツマンマシン

さてホップフィールド模型と同時期に出たのがこれまたノーベル物理学賞を受賞することになったヒントンさんの提案したボルツマンマシンです。
$J_{ij}$にうまく情報を載せれば復元可能ということがホップフィールド模型で確認ができました。
これはいわばエネルギーに情報を直接書き込んだとも言えます。最適解の形を作ったということになります。
一方でとにかく量子アニーリングマシン（シミュレータでもOK）から出力されるものが画像と一致してくれれば良いですよね。
ということで$J_{ij}$や$h_{i}$をうまく調整するパラメータとして「動かす」ことを考えて出力結果をコントロールしましょう。
その出力結果が手書き文字の画像データの傾向に合えば以降量子アニーリングマシンは手書き文字画像データの出力マシンとなります。


In [ ]:
from dwave.system import FixedEmbeddingComposite

このFixedEmbeddingCompositeが埋め込みを指定して固定できるソルバーです。
埋め込みはどんなQUBOを使いたいのかを指定したら自動的にできますので
ホップフィールド模型で用意したQUBOを埋め込むことのできる埋め込みを探しましょう。

In [ ]:
from minorminer import find_embedding

このmninorminerに含まれるfind_embeddingで量子回路に皆さんが用意したQUBOを埋め込む方法を発見してくれます。

In [ ]:
adj = {}
for k in qubo.keys():
  adj[k] = 1
embedding = find_embedding(adj, dw_sampler.edgelist)

まず隣接行列と呼ぶのですが、QUBOに含まれるkeysに従い1を立てます。
これはQUBO行列に数字が埋め込まれているペアを特定するkeysを利用して、どことどこの手を結んで欲しいのかリクエストしています。
その情報に基づきfind_embeddingをします。ここでdw_sampler.edgelistで実際にどのペアが量子回路上に生き残っているのかを参考にして載せていってくれます。

出来上がったembeddingを参考に埋め込みをしましょう。
FixedEmbeddingCompositeで固定したembeddingを実施してsamplerを独自に作成することができます。

In [ ]:
sampler = FixedEmbeddingComposite(dw_sampler, embedding)

これで固定された埋め込みのもとサンプリングを実行するのはいつも通りです。

In [ ]:
num_reads = 10
sampleset = sampler.sample_qubo(qubo, num_reads=num_reads, answer_mode = "raw")

読み出し回数を決めてサンプリングを実行してみました。
大丈夫ですね。実際に出来上がった画像も見てみましょう。

In [ ]:
for k in range(num_reads):
  plt.imshow(sampleset.record[k].sample.reshape(N,N))
  plt.show()

まだこの時点では先ほどのホップフィールド模型の設定が残っています。
それではデータとモデルの出力を合わせるということを考えてみましょう。
相互作用$J_{ij}$を変えると隣との関係、磁場$h_i$を変えると単独の画素の傾向が操作されます。
どういうふうに変えたら良いのかを考えると、
データとモデルのそれぞれ画素値と画素値のペアによる積を考えて、その差を見てみることにしましょう。

やることは1つ1つのペアごとに画素値の積をとります。
これはホップフィールド模型と同じですね。

In [ ]:
#data
qubo_data = np.zeros((N*N,N*N))
for k in range(K):
  qubo_data = qubo_data + np.outer(binary_data[k,:],binary_data[k,:])/K
qubo_data = qubo_data

一方で量子アニーリングマシン（もしくはシミュレータ）側の出力結果でも同じものを計測してみます。

In [ ]:
qubo_model = np.zeros((N*N,N*N))
for k in range(num_reads):
  qubo_model = qubo_model + np.outer(sampleset.record[k][0],sampleset.record[k][0])/ num_reads
qubo_model = qubo_model

これは毎回計算することになることが想定されます。
なぜなら量子アニーリングマシンからの出力を見て、QUBO行列を変更してやり直しをする。
そのやり直していく中で、データの傾向と量子アニーリングマシンが出した傾向を合わせていきます。
このようにしてデータの傾向と合わせたイジング模型やQUBOを作る学習をボルツマンマシン学習、その結果得られたモデルをボルツマンマシンと呼びます。
そこで自作関数という形で、上記の計算を実行できるものを用意しましょう。

In [ ]:
def comp_model(sampleset,num_reads=num_reads):
  qubo_model = np.zeros((N*N,N*N))
  for k in range(num_reads):
    qubo_model = qubo_model + np.outer(sampleset.record[k][0],sampleset.record[k][0])/ num_reads
  return qubo_model

それでは準備をしましょう。
まずは設定パラメータを用意します。
何回更新をするかの回数、更新をする際にどの程度quboを動かすのか。
最初のquboはどんな形にするか。


In [ ]:
Tall = 50
eta = 0.1
qubo = np.random.randn(N**4).reshape(N*N,N*N)
qubo = qubo + qubo.T

次に今日のおさらいです。
量子アニーリングマシンに埋め込みをするので、その埋め込みを決めておきましょう。
その上で埋め込み結果を利用して自作サンプラーを作ります。


In [ ]:
adj = {}
for i in range(N*N):
  for j in range(N*N):
    adj[(i,j)] = 1
embedding = find_embedding(adj, dw_sampler.edgelist)
sampler = FixedEmbeddingComposite(dw_sampler, embedding)

何度か量子アニーリングを実行して、その結果を利用して量子アニーリングマシンからの出力傾向とデータの傾向の差を調べて、調整するプログラムを作ります。

In [ ]:
for t in range(Tall):
  sampleset = sampler.sample_qubo(qubo, num_reads=num_reads, answer_mode = "raw")
  qubo_model = comp_model(sampleset)
  qubo = qubo - eta*(qubo_data - qubo_model)


仕上がりを見てみましょう。
そうすると画像がやはりうまく生成されている様子と
ホップフィールド模型とは異なり、全く同じものではなく少し変化したものが色々と出ていることがわかります。
これはガチっと相互作用行列を定めず微調整をしながらデータの傾向と量子アニーリングマシンの出力結果を合わせた結果です。
これにより量子アニーリング独特の傾向を持ちつつ、データの傾向に合わせた画像生成装置が出来上がり！というわけです。

In [ ]:
for k in range(num_reads):
  res_image = sampleset.record[k][0]
  plt.imshow(res_image.reshape(N,N))
  plt.show()

量子アニーリングマシンが次第に成長していく様子を見て見たいですね。
いくつかの画像データを横並べして、それを繰り返し表示して進化の様子を見てみましょう。

In [ ]:
for t in range(Tall):
  sampleset = sampler.sample_qubo(qubo, num_reads=num_reads, answer_mode = "raw")
  qubo_model = comp_model(sampleset)
  qubo = qubo - eta*(qubo_data - qubo_model)

  for k in range(10):
    plt.subplot(1,10,k+1)
    res_image = sampleset.record[k][0]
    plt.imshow(res_image.reshape(N,N))
  plt.show()